In [25]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 7: Amplification Circuit ---</h2><p>Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of <a href="https://en.wikipedia.org/wiki/Amplifier">amplifiers</a> already installed on the ship.</p>
<p>There are five <span title="As you can see, I know exactly how rockets work.">amplifiers connected in series</span>; each one receives an input signal and produces an output signal.  They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on.  The first amplifier's input value is <code>0</code>, and the last amplifier's output leads to your ship's thrusters.</p>
<pre><code>    O-------O  O-------O  O-------O  O-------O  O-------O
0 -&gt;| Amp A |-&gt;| Amp B |-&gt;| Amp C |-&gt;| Amp D |-&gt;| Amp E |-&gt; (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
</code></pre>
<p>The Elves have sent you some <em>Amplifier Controller Software</em> (your puzzle input), a program that should run on your <a href="5">existing Intcode computer</a>. Each amplifier will need to run a copy of the program.</p>
<p>When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current <em>phase setting</em> (an integer from <code>0</code> to <code>4</code>). Each phase setting is used <em>exactly once</em>, but the Elves can't remember which amplifier needs which phase setting.</p>
<p>The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)</p>
<p>Your job is to <em>find the largest output signal that can be sent to the thrusters</em> by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.</p>
<p>For example, suppose you want to try the phase setting sequence <code>3,1,2,4,0</code>, which would mean setting amplifier <code>A</code> to phase setting <code>3</code>, amplifier <code>B</code> to setting <code>1</code>, <code>C</code> to <code>2</code>, <code>D</code> to <code>4</code>, and <code>E</code> to <code>0</code>. Then, you could determine the output signal that gets sent from amplifier <code>E</code> to the thrusters with the following steps:</p>
<ul>
<li>Start the copy of the amplifier controller software that will run on amplifier <code>A</code>. At its first input instruction, provide it the amplifier's phase setting, <code>3</code>.  At its second input instruction, provide it the input signal, <code>0</code>.  After some calculations, it will use an output instruction to indicate the amplifier's output signal.</li>
<li>Start the software for amplifier <code>B</code>. Provide it the phase setting (<code>1</code>) and then whatever output signal was produced from amplifier <code>A</code>. It will then produce a new output signal destined for amplifier <code>C</code>.</li>
<li>Start the software for amplifier <code>C</code>, provide the phase setting (<code>2</code>) and the value from amplifier <code>B</code>, then collect its output signal.</li>
<li>Run amplifier <code>D</code>'s software, provide the phase setting (<code>4</code>) and input value, and collect its output signal.</li>
<li>Run amplifier <code>E</code>'s software, provide the phase setting (<code>0</code>) and input value, and collect its output signal.</li>
</ul>
<p>The final output signal from amplifier <code>E</code> would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.</p>
<p>Here are some example programs:</p>
<ul>
<li><p>Max thruster signal <em><code>43210</code></em> (from phase setting sequence <code>4,3,2,1,0</code>):</p><pre><code>3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0</code></pre></li>
<li><p>Max thruster signal <em><code>54321</code></em> (from phase setting sequence <code>0,1,2,3,4</code>):</p><pre><code>3,23,3,24,1002,24,10,24,1002,23,-1,23,<br>101,5,23,23,1,24,23,23,4,23,99,0,0</code></pre></li>
<li><p>Max thruster signal <em><code>65210</code></em> (from phase setting sequence <code>1,0,4,3,2</code>):</p><pre><code>3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,<br>1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0</code></pre></li>
</ul>
<p>Try every combination of phase settings on the amplifiers.  <em>What is the highest signal that can be sent to the thrusters?</em></p>
</article>


In [26]:
from more_itertools import first


class IntcodeComputer:
    def __init__(self, s: str) -> None:
        self.state = [int(i) for i in s.split(",")]
        self.reset_state = self.state[:]
        self.ip = 0

    def run(self, inputs=None, reset=True) -> int:
        inputs = inputs if inputs else []

        if reset:
            self.ip = 0

        while True:
            code = self.state[self.ip]
            opcode = code % 100
            code //= 100
            mode_a = code % 10
            code //= 10
            mode_b = code % 10

            if opcode == 1:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]

                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]

                self.ip += 1
                self.state[self.state[self.ip]] = a + b
                self.ip += 1
            elif opcode == 2:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]

                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]

                self.ip += 1
                self.state[self.state[self.ip]] = a * b
                self.ip += 1
            elif opcode == 3:
                self.ip += 1
                self.state[self.state[self.ip]] = inputs.pop(0)
                self.ip += 1
            elif opcode == 4:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]
                self.ip += 1
                return a
            elif opcode == 5:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]
                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]
                if a != 0:
                    self.ip = b
                else:
                    self.ip += 1
            elif opcode == 6:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]
                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]
                if a == 0:
                    self.ip = b
                else:
                    self.ip += 1
            elif opcode == 7:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]
                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]
                self.ip += 1
                self.state[self.state[self.ip]] = 1 if a < b else 0
                self.ip += 1
            elif opcode == 8:
                self.ip += 1
                a = self.state[self.ip] if mode_a else self.state[self.state[self.ip]]
                self.ip += 1
                b = self.state[self.ip] if mode_b else self.state[self.state[self.ip]]
                self.ip += 1
                self.state[self.state[self.ip]] = 1 if a == b else 0
                self.ip += 1
            elif opcode == 99:
                return None
            else:
                raise ValueError("1202 program alarm")

    def reset(self):
        self.state = self.reset_state[:]

    def __repr__(self) -> str:
        return ",".join(str(i) for i in self.state)


tests = [
    {
        "name": "Example 1",
        "phase_setting": [4, 3, 2, 1, 0],
        "program": "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0",
        "expected": 43210,
    },
    {
        "name": "Example 2",
        "phase_setting": [0, 1, 2, 3, 4],
        "program": "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0",
        "expected": 54321,
    },
    {
        "name": "Example 3",
        "phase_setting": [1, 0, 4, 3, 2],
        "program": "3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0",
        "expected": 65210,
    },
]


class AmplificationCircuit:
    def __init__(self, program: str) -> None:
        self.amplifiers = [IntcodeComputer(program) for _ in range(5)]

    def amplify(self, phase_setting: list[int]) -> str:
        out = 0
        for i, amp in enumerate(self.amplifiers):
            out = amp.run([phase_setting[i], out], True)
        return out

    def max_trust(self) -> int:
        return max(self.amplify(phs) for phs in permutations(range(5)))


def test_runner_I(phase_setting, program):
    return AmplificationCircuit(program).amplify(phase_setting)


run_tests_params(test_runner_I, tests)


def test_runner_II(phase_setting, program):
    return AmplificationCircuit(program).max_trust()


run_tests_params(test_runner_II, tests)


Test Example 1 passed, for test_runner_I.
Test Example 2 passed, for test_runner_I.
Test Example 3 passed, for test_runner_I.
Success

Test Example 1 passed, for test_runner_II.
Test Example 2 passed, for test_runner_II.
Test Example 3 passed, for test_runner_II.
Success


In [27]:
with open("../input/day7.txt") as f:
    puzzle = f.read()

print(f"Part I: {AmplificationCircuit(puzzle).max_trust()}")  # 76208 is too low

Part I: 199988


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>199988</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need.  The Elves quickly talk you through rewiring the amplifiers into a <em>feedback loop</em>:</p>
<pre><code>      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+-&gt;| Amp A |-&gt;| Amp B |-&gt;| Amp C |-&gt;| Amp D |-&gt;| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
</code></pre>
<p>Most of the amplifiers are connected as they were before; amplifier <code>A</code>'s output is connected to amplifier <code>B</code>'s input, and so on. <em>However,</em> the output from amplifier <code>E</code> is now connected into amplifier <code>A</code>'s input. This creates the feedback loop: the signal will be sent through the amplifiers <em>many times</em>.</p>
<p>In feedback loop mode, the amplifiers need <em>totally different phase settings</em>: integers from <code>5</code> to <code>9</code>, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.</p>
<p>Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.</p>
<p>All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a <code>0</code> signal is sent to amplifier <code>A</code>'s input <em>exactly once</em>.</p>
<p>Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier <code>E</code> is sent to the thrusters. Your job is to <em>find the largest output signal that can be sent to the thrusters</em> using the new phase settings and feedback loop arrangement.</p>
<p>Here are some example programs:</p>
<ul>
<li><p>Max thruster signal <em><code>139629729</code></em> (from phase setting sequence <code>9,8,7,6,5</code>):</p><pre><code>3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,<br>27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5</code></pre></li>
<li><p>Max thruster signal <em><code>18216</code></em> (from phase setting sequence <code>9,7,8,5,6</code>):</p><pre><code>3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,<br>-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,<br>53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10</code></pre></li>
</ul>
<p>Try every combination of the new phase settings on the amplifier feedback loop.  <em>What is the highest signal that can be sent to the thrusters?</em></p>
</article>

</main>


In [42]:
from copy import deepcopy
from more_itertools import take


class FeedBackAmplificationCircuit:
    def __init__(self, program: str) -> None:
        self.amplifiers = [IntcodeComputer(program) for _ in range(5)]

    def amplify(self, phase_setting: list[int]) -> str:
        out = prev = 0

        for i, amp in enumerate(self.amplifiers):
            out = amp.run([phase_setting[i], out])
            if out is None:
                return prev
            prev = out

        while True:
            for amp in self.amplifiers:
                out = amp.run([out], False)
                if out is None:
                    return prev
                prev = out

    def max_trust(self) -> int:
        return max(deepcopy(self).amplify(phs) for phs in permutations(range(5, 10)))


tests = [
    {
        "name": "Example 1",
        "program": "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5",
        "phase_setting": [9, 8, 7, 6, 5],
        "expected": 139629729,
    },
    {
        "name": "Example 2",
        "program": "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10",
        "phase_setting": [9, 7, 8, 5, 6],
        "expected": 18216,
    },
]


def test_runner_III(phase_setting, program):
    return FeedBackAmplificationCircuit(program).amplify(phase_setting)


run_tests_params(test_runner_III, tests)


def test_runner_IV(phase_setting, program):
    return FeedBackAmplificationCircuit(program).max_trust()


run_tests_params(test_runner_IV, tests)  # don't finish programs odd


Test Example 1 passed, for test_runner_III.
Test Example 2 passed, for test_runner_III.
Success

Test Example 1 passed, for test_runner_IV.
Test Example 2 passed, for test_runner_IV.
Success


In [32]:
print(f"Part II: {FeedBackAmplificationCircuit(puzzle).max_trust()}")

Part II: 17519904


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>17519904</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
